In [49]:
import datetime
import numpy as np
import os
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt



In [50]:
#Created a 07/23/2021 
#Last opened 
print(f'Last edited: {datetime.datetime.now()}')

Last edited: 2021-12-13 16:32:32.580121


Get the background first <br>
Then shift so that all of them start at the same point  <br>
Then do the scaling 

In [54]:
# load constraints from the data and save all the means and 2nd moments, then save all the required points 
## saving Data of IGF
filepath  = 'Data/IGF/EXP129_RAW.xlsx'

# df = pd.read_excel(xls, 'Sheet2' ,header=None)
# bg_array = df.to_numpy()[:,1]

xls = pd.ExcelFile(filepath)
Data_Dict = defaultdict(lambda: 'Not present')
Sheet_labels = ['Sheet2' ,'0pM','10pM','15pm','20pM','25pM', '50pM', '250pM']
# conc
nSheets = len(Sheet_labels)
for i in range(nSheets): 
    sl = Sheet_labels[i]
    df = pd.read_excel(xls, sl ,header=None)
    data = df.to_numpy()
    Data_Dict[sl] = data
    print(f'saved {sl} to dictionary')


saved Sheet2 to dictionary
saved 0pM to dictionary
saved 10pM to dictionary
saved 15pm to dictionary
saved 20pM to dictionary
saved 25pM to dictionary
saved 50pM to dictionary
saved 250pM to dictionary


In [86]:
bg_array = Data_Dict['Sheet2'][:,1]
bg_matrix = np.transpose(np.tile(bg_array, (300,1)))
# conc
mean_bgrem = defaultdict(lambda: 'not present')
bgg = 0 
ncc = 0
bg_removed_dict = defaultdict(lambda: 'Not present')
conc_labels = ['10pM','15pm','20pM','50pM']
for key in conc_labels: 
    print(key)
    matrix = Data_Dict[key][:,1:]
    ncells = matrix.shape[1]
    bg_matrix = np.transpose(np.tile(bg_array, (ncells,1)))

    data = matrix - bg_matrix
    bgg += np.sum(data[0,:])
    ncc += len(data[0,:]) 
    bg_removed_dict[key] = data
    mean_bgrem[key] = np.mean(data, axis = 1)
    print(mean_bgrem[key].shape)

Mean_all_cells_ov_all_cells = bgg/ncc
print(Mean_all_cells_ov_all_cells)

10pM
(61,)
15pm
(61,)
20pM
(61,)
50pM
(61,)
22530.951948376358


In [87]:
conc_labels

['10pM', '15pm', '20pM', '50pM']

In [88]:
n_foxO = 710*2/3 
au_foxO = 22530.951948376358
# scale factor 
sc = n_foxO/au_foxO


drift = np.zeros(len(conc_labels))
# data_bg_sh_sc = defaultdict(lambda: 'not present')
means_bg_sh_sc = defaultdict(lambda: 'not present')
sm_bg_sh_sc = defaultdict(lambda: 'not present')


for i in range(len(drift)): 
    key = conc_labels[i]
    drift[i] = Mean_all_cells_ov_all_cells - mean_bgrem[key][0]
    data = bg_removed_dict[key] 
    data += drift[i]
    
    means_bg_sh[key ] = np.mean(data, axis = 1)
    
    data2 = data * sc 
    
    
    means_bg_sh_sc[key] = np.mean(data2, axis =1)
    
    sm_bg_sh_sc[key] = np.mean(data2**2 , axis =1)
    

    
    
    
#     means_bg_sh[key ] = np.mean(data, axis = 1)
    
    
    
    
    
    

In [101]:
sm_bg_sh_sc

defaultdict(<function __main__.<lambda>()>,
            {'10pM': array([228871.37514697, 190161.07167463, 210515.38738396, 211915.66155146,
                    211707.29795449, 205631.16443147, 202978.3085275 , 199199.93717261,
                    195454.58971385, 191912.3558708 , 188428.21562382, 185269.99018773,
                    181947.87535975, 178930.42781927, 176869.19361498, 174209.21366677,
                    171684.90475895, 170202.63851865, 168312.65084522, 166713.02840032,
                    165514.94135594, 164405.14278667, 163909.86694219, 163445.73816543,
                    163247.92370094, 163213.61631202, 162369.68276851, 162384.28783418,
                    162347.26696382, 162670.17941838, 162745.85950338, 163279.98397235,
                    163256.86081286, 163445.21969947, 163733.12228732, 164510.50706471,
                    164010.04137496, 164365.22504606, 165025.61828054, 164588.30052729,
                    164573.09546939, 165243.9999571 , 165691.7570147

In [97]:
drift

array([-380.47758651, -129.27138496,  -15.29805162,  526.31528171])

In [ ]:
## make the shift 
# Use this for the caling
# au_fox0 = 22121.924935744886


### Saving the data with bg removal, shift added and scaling .

### Getting the constraints and saving them 

In [33]:
time_points = Data_Dict[key][:,0]
time_points

array([  0,   3,   6,   9,  12,  15,  18,  21,  24,  27,  30,  33,  36,
        39,  42,  45,  48,  51,  54,  57,  60,  63,  66,  69,  72,  75,
        78,  81,  84,  87,  90,  93,  96,  99, 102, 105, 108, 111, 114,
       117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147, 150, 153,
       156, 159, 162, 165, 168, 171, 174, 177, 180])

In [98]:
time_values = [0,6,12,24,45,60,90]
times_constrained = np.array(time_values) #minutes 
idx_list = np.array(times_constrained/3, dtype=int)

In [99]:
times_constrained

array([ 0,  6, 12, 24, 45, 60, 90])

In [100]:
idx_list

array([ 0,  2,  4,  8, 15, 20, 30])

In [43]:
# constraints_means_dict = defaultdict(lambda: 'not present')
conc_list = [10,15,20,50]
conc_labels = ['10pM','15pm','20pM','50pM']
labels_arr = []
cons_arr = np.zeros(len(conc_list)* len(time_values)*2)
k=0
titles_list = []
means_list = []
sm_list = []
for i in range(len(conc_list)):
    for j in range(len(time_values)):
        title = f'{conc_list[i]}_pM_{time_values[j]}_min'
        # save the means
        key = conc_labels[i]
        means_list.append(Means_dict[key][idx_list[j]])
        # save the second moments 
        sm_list.append(SecondMoment_dict[key][idx_list[j]])
        # save the titles 
        labels_arr.append(title)


In [46]:
np.log10(sm_list)

array([5.35957951, 5.32327137, 5.3257234 , 5.29103295, 5.24105747,
       5.21882323, 5.21149586, 5.35985122, 5.34158939, 5.31441612,
       5.23523097, 5.16475685, 5.13135201, 5.12561225, 5.35918753,
       5.2972882 , 5.27947457, 5.17351093, 5.05803982, 5.02783516,
       5.02447215, 5.36255037, 5.28418533, 5.19484007, 4.89863601,
       4.71101053, 4.70092906, 4.71853669])

In [61]:
labels_arr

['10_pM_0_min',
 '10_pM_6_min',
 '10_pM_12_min',
 '10_pM_24_min',
 '10_pM_45_min',
 '10_pM_60_min',
 '10_pM_90_min',
 '15_pM_0_min',
 '15_pM_6_min',
 '15_pM_12_min',
 '15_pM_24_min',
 '15_pM_45_min',
 '15_pM_60_min',
 '15_pM_90_min',
 '20_pM_0_min',
 '20_pM_6_min',
 '20_pM_12_min',
 '20_pM_24_min',
 '20_pM_45_min',
 '20_pM_60_min',
 '20_pM_90_min',
 '50_pM_0_min',
 '50_pM_6_min',
 '50_pM_12_min',
 '50_pM_24_min',
 '50_pM_45_min',
 '50_pM_60_min',
 '50_pM_90_min']

In [62]:
dictionary = {'labels': labels_arr, 'array':cons_arr}

In [63]:
np.save('Cons_1208_Zero2ndMoments.npy', dictionary) 



In [65]:
np.save('/Volumes/hodaakl/A5MCMC_IGF_FoxO/Arrays_for_max_ent/Cons_1208_Zero2ndMoments.npy', dictionary) 



In [58]:
dictionary

{'labels': ['10_pM_0_min',
  '10_pM_6_min',
  '10_pM_12_min',
  '10_pM_24_min',
  '10_pM_45_min',
  '10_pM_60_min',
  '10_pM_90_min',
  '15_pM_0_min',
  '15_pM_6_min',
  '15_pM_12_min',
  '15_pM_24_min',
  '15_pM_45_min',
  '15_pM_60_min',
  '15_pM_90_min',
  '20_pM_0_min',
  '20_pM_6_min',
  '20_pM_12_min',
  '20_pM_24_min',
  '20_pM_45_min',
  '20_pM_60_min',
  '20_pM_90_min',
  '50_pM_0_min',
  '50_pM_6_min',
  '50_pM_12_min',
  '50_pM_24_min',
  '50_pM_45_min',
  '50_pM_60_min',
  '50_pM_90_min'],
 'array': array([473.3266724 , 454.10860632, 454.92797602, 436.5247793 ,
        410.89845591, 399.71709395, 396.37161368, 473.3266724 ,
        463.36560404, 448.82085023, 408.61048502, 374.72214861,
        359.57389104, 357.00817722, 473.3266724 , 440.84614128,
        431.31182823, 380.0354967 , 330.08426843, 317.88807432,
        316.34335599, 473.3266724 , 432.45472093, 389.35206541,
        273.95826841, 219.50611675, 216.90903078, 221.13188648])}

In [18]:
# n_foxO = 710*2/3 
# au_foxO = 22530.951948376358
# # scale factor 
# sc = n_foxO/au_foxO
# # sc

# # initialize data matrix that contains mean values 
# means_matrix = np.zeros((len(Means_dict),data.shape[0]))
# i=0
# for key in Means_dict: 
#     means_matrix[i,:] = Means_dict[key]* sc
#     i+=1 
    


In [33]:
# these have 61 points in each array, for 61 measurements 
SecondMoment_dict = dict(Means_dict)
np.save('Arrays_for_max_ent/SecondMoment_Dictionary.npy', SecondMoment_dict)#, allow_pickle=True) 

In [34]:
# these have 61 points in each array, for 61 measurements 
Var_dict = dict(Var_dict)
np.save('Arrays_for_max_ent/Variance_Dictionary.npy', Var_dict)#, allow_pickle=True) 

In [35]:
np.load('Arrays_for_max_ent/Means_Dictionary.npy', allow_pickle=True)

array({'0pM': array([25489.34666667, 22560.97666667, 24270.98333333, 24165.06      ,
       24214.46666667, 24113.4       , 24133.50333333, 24191.44333333,
       24239.72      , 24300.82      , 24307.35666667, 24378.20333333,
       24378.68      , 24454.94      , 24452.21666667, 24502.26      ,
       24531.81666667, 24562.89333333, 24566.34      , 24585.95      ,
       24631.36      , 24629.25333333, 24668.28      , 24675.84333333,
       24706.13666667, 24717.19666667, 24696.08666667, 24704.83333333,
       24721.64666667, 24773.01333333, 24815.79666667, 24802.21666667,
       24807.68      , 24816.63666667, 24859.97      , 24886.76333333,
       24925.70666667, 24942.10666667, 24960.27      , 24972.66      ,
       25040.71666667, 25031.67      , 25029.28      , 25051.52666667,
       25075.49666667, 25083.17      , 25111.35333333, 25124.02333333,
       25132.53333333, 25175.16333333, 25166.63666667, 25162.2       ,
       25206.18333333, 25228.42666667, 25229.37666667, 25244.85

In [7]:
# Making the constrained array 
# for all the concentrations except 0 pM
# 0, 6, 15, 21, 51, 126 min
times_constrained = np.array([0, 6, 12, 25, 45,60, 90]) #minutes 
idx_list = np.array(times_constrained/3, dtype=int)

In [22]:
# Now making the constrained dictionary 

array([ 0,  2,  5,  7, 17, 42])

In [8]:
Constraints_Dict = defaultdict(lambda:'NotPresent')
Constraints_Dict['Mean']= {}
Constraints_Dict['Var'] = {}
Constraints_Dict['SecMoment'] = {}


Constraints_Dict['Times(min)'] = times_constrained

for i in range(len(Sheet_labels) -1): 
    conc = Sheet_labels[i+1]
    if conc == '15pm':
        con_key = '15pM'
    else:
        con_key = conc
    Constraints_Dict['Mean'][con_key] = Means_dict[conc][idx_list]
    Constraints_Dict['SecMoment'][con_key] = SecondMoment_dict[conc][idx_list]
    Constraints_Dict['Var'][con_key] = Var_dict[conc][idx_list]

In [6]:
Constraints_Dict

defaultdict(<function __main__.<lambda>()>,
            {'Mean': {'10pM': array([26629.13953488, 25580.74086379, 25612.27906977, 24624.97009967,
                     23366.93023256, 22810.94352159, 22608.14950166]),
              '15pM': array([26377.93333333, 25770.17333333, 25070.37      , 23045.02666667,
                     21393.71      , 20648.89666667, 20483.22      ]),
              '20pM': array([26263.96      , 24584.26      , 24122.95666667, 21570.86666667,
                     19154.94666667, 18550.65333333, 18433.57666667]),
              '25pM': array([25028.07      , 24102.68333333, 23689.69666667, 20595.28333333,
                     17104.38      , 16043.79666667, 15474.88666667]),
              '50pM': array([25722.34666667, 23643.21      , 21584.03333333, 15979.91333333,
                     13349.74666667, 13202.37666667, 13359.84      ]),
              '250pM': array([25364.01666667, 22210.54      , 17204.36      , 12114.49      ,
                     11753.6933333

In [9]:
times_constrained

array([ 0,  6, 12, 25, 45, 60, 90])

In [20]:
n_foxO = 710*2/3 
au_foxO = 25299.566666666666
# scale factor 
sc = n_foxO/au_foxO
sc

0.01870914785101721

In [22]:
LigandCon = [10,15,20,25,50,250]
nTcon = len(times_constrained)
nLcon = len(LigandCon)
nCons = nTcon*nLcon*2

Cons_means = []
Cons_sm    = []
Cons_names = []
# store means first 

for ligand in LigandCon: 
    key = f'{ligand}pM'
    ii = 0
    for timecon in times_constrained: 
        timestring = f'{timecon}min'
        title = f'{key}_{timestring}'
        Cons_names.append(title)
        Cons_means.append(Constraints_Dict['Mean'][key][ii])
        Cons_sm.append(Constraints_Dict['SecMoment'][key][ii])
        ii+=1 
        

# for key in Constraints_Dict['Mean']: 
#     Cons_Arr[ii:(ii+6)] = Constraints_Dict['Mean'][key]
#     ii+=6
Cons_means = np.asarray(Cons_means)*sc
Cons_sm = np.asarray(Cons_sm)*(sc**2)

In [23]:
con_dic = { 'labels' : [], 'array' : []}
i = 0 
for tit in Cons_names: 
    con_dic['labels'].append(f'mean_{tit}')
    con_dic['array'].append(Cons_means[i])
    i+=1 

    

i = 0 
for tit in Cons_names: 
    con_dic['labels'].append(f'sm_{tit}')
    con_dic['array'].append(Cons_sm[i])
    i+=1 


# 
con_dic['array']= np.asarray(con_dic['array'])
con_dic

{'labels': ['mean_10pM_0min',
  'mean_10pM_6min',
  'mean_10pM_12min',
  'mean_10pM_25min',
  'mean_10pM_45min',
  'mean_10pM_60min',
  'mean_10pM_90min',
  'mean_15pM_0min',
  'mean_15pM_6min',
  'mean_15pM_12min',
  'mean_15pM_25min',
  'mean_15pM_45min',
  'mean_15pM_60min',
  'mean_15pM_90min',
  'mean_20pM_0min',
  'mean_20pM_6min',
  'mean_20pM_12min',
  'mean_20pM_25min',
  'mean_20pM_45min',
  'mean_20pM_60min',
  'mean_20pM_90min',
  'mean_25pM_0min',
  'mean_25pM_6min',
  'mean_25pM_12min',
  'mean_25pM_25min',
  'mean_25pM_45min',
  'mean_25pM_60min',
  'mean_25pM_90min',
  'mean_50pM_0min',
  'mean_50pM_6min',
  'mean_50pM_12min',
  'mean_50pM_25min',
  'mean_50pM_45min',
  'mean_50pM_60min',
  'mean_50pM_90min',
  'mean_250pM_0min',
  'mean_250pM_6min',
  'mean_250pM_12min',
  'mean_250pM_25min',
  'mean_250pM_45min',
  'mean_250pM_60min',
  'mean_250pM_90min',
  'sm_10pM_0min',
  'sm_10pM_6min',
  'sm_10pM_12min',
  'sm_10pM_25min',
  'sm_10pM_45min',
  'sm_10pM_60min',
 

In [24]:
# Save that dictionary 

# dictionary = {'hello':'world'}
np.save('Cons_1117.npy', con_dic) 

# # Load
# read_dictionary = np.load('my_file.npy',allow_pickle='TRUE').item()
# print(read_dictionary['hello']) # displays "world"




In [25]:
read_dictionary = np.load('Cons_1117.npy',allow_pickle='TRUE').item()

In [27]:
read_dictionary['array']

array([4.98208509e+02, 4.78593863e+02, 4.79183916e+02, 4.60712206e+02,
       4.37175353e+02, 4.26773315e+02, 4.22979212e+02, 4.93508655e+02,
       4.82137983e+02, 4.69045259e+02, 4.31152811e+02, 4.00258083e+02,
       3.86323261e+02, 3.83223591e+02, 4.91376311e+02, 4.59950555e+02,
       4.51319963e+02, 4.03572534e+02, 3.58372729e+02, 3.47066916e+02,
       3.44876511e+02, 4.68253862e+02, 4.50940666e+02, 4.43214037e+02,
       3.85320201e+02, 3.20008374e+02, 3.00165764e+02, 2.89521943e+02,
       4.81243187e+02, 4.42344312e+02, 4.03818871e+02, 2.98970561e+02,
       2.49762384e+02, 2.47005217e+02, 2.49951222e+02, 4.74539138e+02,
       4.15540277e+02, 3.21878915e+02, 2.26651785e+02, 2.19901586e+02,
       2.17709124e+02, 2.15389564e+02, 2.52040001e+05, 2.32458260e+05,
       2.33377969e+05, 2.16137920e+05, 1.95378270e+05, 1.86684626e+05,
       1.83376715e+05, 2.47492669e+05, 2.36319952e+05, 2.23826533e+05,
       1.89794603e+05, 1.64742474e+05, 1.54022961e+05, 1.51686731e+05,
      